In [3]:
import ott
import numpy as np
from entot.models.model import OTFlowMatching
from entot.nets.nets import MLP_vector_field, MLP_bridge
import sys
import os



source_data_train = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_source_samples_0.1_32.npy")
target_data_train = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_target_samples_0.1_32.npy")
source_data_test = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_source_samples_0.1_32.npy")
target_data_test = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_target_samples_0.1_32.npy")
epsilon = 0.1
seed=0


ot_solver=ott.solvers.linear.sinkhorn.Sinkhorn()
neural_net = MLP_vector_field(target_data_train.shape[1], latent_embed_dim = 256, num_layers=8, n_frequencies=256)
bridge_net = MLP_bridge(target_data_train.shape[1], target_data_train.shape[1], bridge_type="mean")

otfm = OTFlowMatching(neural_net, iterations=10_000,  bridge_net=bridge_net,  ot_solver=ot_solver, epsilon=epsilon, input_dim=source_data_train.shape[1], output_dim=target_data_train.shape[1], k_noise_per_x=1,  seed=seed)
otfm(source_data_train, target_data_train, 1024, 1024)

res = [None] * 2

#for i in range(2):
#    res[i] = otfm.transport(source_data_test, seed=i)[0][0,...]

#samples = np.asarray(res)




2023-09-22 09:09:34.889834: W external/xla/xla/service/gpu/nvptx_compiler.cc:698] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.2.128). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
100%|██████████| 10000/10000 [36:56<00:00,  4.51it/s]


In [4]:
res = [None] * 2

for i in range(2):
    res[i] = otfm.transport(source_data_test, seed=i)[0][0,...]

samples = np.asarray(res)

In [5]:
opt_coupling = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_optimal_samples_0.1_32.npy")

In [8]:
compute_BW_UVP_by_gt_samples(opt_coupling, np.concatenate((source_data_train, np.squeeze(samples[0,...])), axis=1))

13.768306681742834

In [17]:
compute_BW_UVP_by_gt_samples(opt_coupling, np.concatenate((source_data_train, np.squeeze(samples[0,...])), axis=1))

14.085117601788403

In [4]:
otfm.t_offset

0.000975609756097561

In [6]:
n_samples=1024

t = (jax.random.uniform(jax.random.PRNGKey(0), (1,)) + jnp.arange(n_samples) / n_samples) % (1 - otfm.t_offset)

In [7]:
t.shape

(1024,)

In [8]:
jax.random.uniform(jax.random.PRNGKey(0), (1024,)).shape

(1024,)

In [22]:
res = [None] * 2

for i in range(2):
    res[i] = otfm.transport(source_data_test, seed=i)[0][0,...]

samples = np.asarray(res)

In [3]:
t = (jax.random.uniform(jax.random.PRNGKey(0), (1024,)))

In [4]:
t

Array([0.09477913, 0.28464484, 0.35750294, ..., 0.9188044 , 0.5129981 ,
       0.53420603], dtype=float32)

In [23]:
samples.shape

(2, 5000, 32)

In [24]:
opt_coupling = np.load("/home/icb/dominik.klein/git_repos/genot_benchmarks/input_data/gaussian_data/gaussian_optimal_samples_0.1_32.npy")

In [25]:
opt_coupling.shape

(5000, 64)

In [26]:
compute_BW_UVP_by_gt_samples(opt_coupling, np.concatenate((source_data_train, np.squeeze(samples[0,...])), axis=1))

13.60172694571213

In [7]:
# this code is based on https://github.com/PetrMokrov/Energy-guided-Entropic-OT/tree/main
import os

import numpy as np
from scipy.linalg import sqrtm, inv

import gc

def symmetrize(X):
    return np.real((X + X.T) / 2)


def compute_BW_UVP_with_gt_stats(model_samples, true_samples_mu, true_samples_covariance):
    model_samples_covariance = np.cov(model_samples.T)
    model_samples_mu = model_samples.mean(axis=0)
    model_samples_covariance_sqrt = symmetrize(sqrtm(model_samples_covariance))
    
    mu_term = 0.5*((true_samples_mu - model_samples_mu)**2).sum()
    covariance_term = (
        0.5*np.trace(model_samples_covariance) + 
        0.5*np.trace(true_samples_covariance) -
        np.trace(symmetrize(sqrtm(model_samples_covariance_sqrt@true_samples_covariance@model_samples_covariance_sqrt)))
    )

    BW = mu_term + covariance_term
    BW_UVP = 100*(BW/(0.5*np.trace(true_samples_covariance)))
        
    return BW_UVP


def compute_BW_UVP_by_gt_samples(model_samples, true_samples):
    true_samples_covariance = np.cov(true_samples.T)
    true_samples_mu = true_samples.mean(axis=0)
        
    return compute_BW_UVP_with_gt_stats(model_samples, true_samples_mu, true_samples_covariance)

def symmetrize(X):
    return np.real((X + X.T) / 2)


def get_D_sigma(covariance_0, covariance_T, epsilon):
    shape = covariance_0.shape[0]
    
    covariance_0_sqrt = symmetrize(sqrtm(covariance_0))
    return symmetrize(sqrtm(4*covariance_0_sqrt@covariance_T@covariance_0_sqrt + (epsilon**2)*np.eye(shape)))


def get_C_sigma(covariance_0, D_sigma, epsilon):
    shape = covariance_0.shape[0]
    
    covariance_0_sqrt = symmetrize(sqrtm(covariance_0))
    covariance_0_sqrt_inv = inv(covariance_0_sqrt)
    
    return 0.5*(covariance_0_sqrt@D_sigma@covariance_0_sqrt_inv - epsilon*np.eye(shape))


def get_mu_t(t, mu_0, mu_T):
    return (1 - t)*mu_0 + t*mu_T


def get_covariance_t(t, covariance_0, covariance_T, C_sigma, epsilon):
    shape = covariance_0.shape[0]
    
    return (
        ((1-t)**2)*covariance_0 + (t**2)*covariance_T + 
        t*(1-t)*(C_sigma+C_sigma.T) + epsilon*t*(1-t)*np.eye(shape)
    )


def get_conditional_covariance_t(t, covariance_0, covariance_T, C_sigma, epsilon):
    shape = covariance_0.shape[0]
    
    covariance_0_inv = inv(covariance_0)
    
    return (
        (t**2)*(covariance_T - C_sigma.T@covariance_0_inv@C_sigma) + 
        epsilon*t*(1-t)*np.eye(shape)
    )


def get_conditional_mu_t(x0, mu_0, mu_T, t, covariance_0, C_sigma, epsilon):
    shape = covariance_0.shape[0]
    
    covariance_0_inv = inv(covariance_0)
    
    return (
        (1-t)*x0 + t*(mu_T + C_sigma.T@covariance_0_inv@(x0[:, None] - mu_0[:, None]))
    )


def get_optimal_plan_covariance(covariance_0, covariance_T, eps):
    covariance_0 = covariance_0
    covariance_T = covariance_T
    
    D_sigma = get_D_sigma(covariance_0, covariance_T, eps)
    C_sigma = get_C_sigma(covariance_0, D_sigma, eps)
    
    size = covariance_0.shape[0]
    optimal_plan_covariance = np.zeros((2*size, 2*size))
    
    optimal_plan_covariance[:size, :size] = covariance_0
    optimal_plan_covariance[size:, size:] = covariance_T
    
    optimal_plan_covariance[:size, size:] = C_sigma
    optimal_plan_covariance[size:, :size] = C_sigma.T
    
    return optimal_plan_covariance